# Serverless Web Apps using Amazon DynamoDB

### AMAZON DYNAMODB
Amazon DynamoDB es un servicio de base de datos NoSQL ágil y flexible para todas las aplicaciones que necesiten una latencia constante en mili segundos de un solo dígito a cualquier escala. Se trata de una base de datos completamente administrada en la nube, compatible con modelos de almacén de valor de clave y de documentos. Su modelo de datos flexible y su desempeño de confianza lo convierten en un complemento perfecto para aplicaciones móviles, web, de juegos, de tecnología publicitaria y de IoT, entre otras.

Puede utilizar Amazon DynamoDB para crear una tabla de base de datos capaz de almacenar y recuperar cualquier cantidad de datos, así como de satisfacer cualquier nivel de tráfico de solicitudes. Amazon DynamoDB distribuye automáticamente los datos y el tráfico de la tabla en una cantidad suficiente de servidores como para manejar la capacidad de solicitudes especificada por el cliente y la cantidad de datos almacenados, al mismo tiempo que mantiene un rendimiento uniforme y rápido.

#### TERMINOLOGÍA DE DYNAMODB
* **Tables (Tablas)** Al igual que otros sistemas de administración de bases de datos, DynamoDB almacena los datos en tablas. Una tabla es un conjunto de datos. Por ejemplo, en este laboratorio, creará una tabla denominada SuperMission, donde almacenará la información de las misiones. También puede tener una tabla llamada People (Personas) para almacenar datos sobre amigos, familiares o cualquier otra persona de su interés, o una tabla llamada Cars (Automóviles) para almacenar información sobre los vehículos que conducen las personas.

* **Items (Elementos)** Cada tabla contiene varios elementos. Un elemento es un grupo de atributos que se identifica de forma única entre todos los demás elementos. En una tabla People (Personas), cada elemento representa una persona. En una tabla Cars (Automóviles), cada elemento representa un automóvil. Los elementos son similares en muchos aspectos a las filas, los registros o las listas ordenadas de otros sistemas de bases de datos relacionales. En DynamoDB, no existe ningún límite con respecto al número de elementos que puede almacenarse en una tabla.

* **Attributes (Atributos)** Cada elemento se compone de uno o más atributos. Un atributo es un componente fundamental de los datos que no es necesario seguir dividiendo. En este laboratorio, creará un elemento Mission (Misión) con atributos como SuperHero (Superhéroe), MissionStatus (Estado de la misión), Villain1 (Villano 1), Villain2 (Villano 2), Villain3 (Villano 3) y SecretIdentity (Identidad secreta). Otro ejemplo sería un elemento Department (Departamento) que podría tener atributos, como DepartmentID (ID de departamento), Name (Nombre), Manager (Gerente), etc. Un elemento de una tabla People (Personas) podría contener atributos como PersonID (ID de la persona), LastName (Apellido), FirstName (Nombre), etc. Los atributos de DynamoDB se parecen en muchos aspectos a los campos o las columnas en otros sistemas de administración de bases de datos.

* **Primary Key (Clave principal)** Cuando crea una tabla, además del nombre de la tabla, debe especificar una clave principal para esta. Como en otras bases de datos, una clave principal en DynamoDB identifica de forma única a cada elemento de la tabla, de modo que no hay dos elementos que tengan la misma clave. Cuando agregue, actualice o elimine un elemento de la tabla, debe especificar los valores de los atributos de la clave principal de ese elemento. Los valores de clave son obligatorios; no puede omitirlos. DynamoDB admite dos tipos diferentes de claves principales: Partition Key (Clave de partición) y Partition Key and Sort Key (Clave de partición y clave de ordenación).

* **Secondary Indexes (Índices secundarios)** En DynamoDB, puede leer datos en una tabla proporcionando los valores de los atributos de la clave principal. Si desea leer los datos utilizando atributos no clave, puede utilizar un índice secundario para hacerlo. Después de crear un índice secundario en una tabla, puede leer datos del índice de la misma manera que lo hace desde la tabla. Al usar índices secundarios, sus aplicaciones pueden usar muchos patrones de consulta diferentes, además de acceder a los datos mediante valores de clave principal.

### DOCUMENTACIÓN DE AWS IDENTITY AND ACCESS MANAGEMENT
AWS Identity and Access Management (IAM) es un servicio web que permite a los clientes de Amazon Web Services (AWS) administrar los usuarios y los permisos de estos en AWS. El servicio está dirigido a organizaciones con varios usuarios o sistemas en la nube que utilizan productos de AWS como Amazon DynamoDB, AWS Lambda y AWS Management Console. Con IAM, puede administrar de forma centralizada a los usuarios, las credenciales de seguridad, como las claves de acceso, y los permisos que controlan a qué recursos de AWS pueden acceder los usuarios. Para obtener más información, consulte https://aws.amazon.com/iam/.

### AWS LAMBDA
AWS Lambda es un servicio de cómputo que brinda capacidad de cómputo de tamaño modificable en la nube con el fin de facilitar la informática a escala web para los desarrolladores. Cargue su código en AWS Lambda y este servicio se encargará de aprovisionar y administrar los servidores que utiliza para ejecutar el código. AWS Lambda admite varios lenguajes de codificación: Node.js, Java o Python. Puede utilizar AWS Lambda de dos formas:

* Como servicio de cómputo basado en eventos, AWS Lambda ejecuta el código en respuesta a eventos, como la carga de archivos de imagen, como verá en este laboratorio.
* Como servicio de cómputo para ejecutar el código en respuesta a solicitudes HTTP mediante Amazon API Gateway o llamadas a la API

AWS Lambda le transfiere los beneficios financieros de la escala de Amazon. Lambda solo ejecuta su código cuando es necesario y escala de forma automática, desde unas pocas solicitudes por día hasta miles por segundo. También facilita la creación de desencadenadores de procesamiento de datos para servicios de AWS como Amazon S3 y Amazon DynamoDB, el procesamiento de streaming de datos almacenados en Amazon Kinesis o la creación de su propio backend que opere según la escala, el rendimiento y la seguridad de AWS.

## Crear la tabla de DynamoDB

In [65]:
import boto3
import json
from zipfile import ZipFile
from pprint import pprint

dynamo = boto3.client('dynamodb')
lambda_client = boto3.client('lambda')
iam = boto3.client('iam')

table_name = 'SuperMissionTable'

In [9]:
# Create dynamodb table
response = dynamo.create_table(
    TableName=table_name,
    AttributeDefinitions=[
        {
            'AttributeName': 'SuperHero',
            'AttributeType': 'S'
        },
    ],
    KeySchema=[
        {
            'AttributeName': 'SuperHero',
            'KeyType': 'HASH'
        },
    ],
    ProvisionedThroughput={
        'ReadCapacityUnits': 5,
        'WriteCapacityUnits': 5
    }
)

# Wait until the table exists.
dynamo.get_waiter('table_exists').wait(TableName=table_name)

In [63]:
# Put elements to table
dynamo.put_item(
    TableName=table_name,
    Item={
        'SuperHero': {'S': 'Superman'},
        'SuperMission': {'S': 'Save the world'},
        'SuperPower': {'S': 'Super strength'},
        'SuperWeapon': {'S': 'Laser eyes'},
        'SuperWeakness': {'S': 'Kryptonite'},
        'Villain1': {'S': 'Lex Luthor'},
        'Villain2': {'S': 'Brainiac'},
        'Villain3': {'S': 'Darkseid'},
        'Villain4': {'S': 'Bizarro'},
    }
)
dynamo.put_item(
    TableName=table_name,
    Item={
        'SuperHero': {'S': 'Batman'},
        'SuperMission': {'S': 'Save Gotham City'},
        'SuperPower': {'S': 'Super intelligence'},
        'SuperWeapon': {'S': 'Utility belt'},
        'Villain1': {'S': 'Joker'},
        'Villain2': {'S': 'Two-Face'},
        'Villain3': {'S': 'Penguin'},

    }
)
dynamo.put_item(
    TableName=table_name,
    Item={
        'SuperHero': {'S': 'Spiderman'},
        'SuperMission': {'S': 'Catch the bad guys'},
        'SuperPower': {'S': 'Super agility'},
        'SuperWeapon': {'S': 'Spider web'},
        'Villain1': {'S': 'Green Goblin'},
        'Villain2': {'S': 'Venom'},
    }
)
dynamo.put_item(
    TableName=table_name,
    Item={
        'SuperHero': {'S': 'Ironman'},
        'SuperMission': {'S': 'Save the world'},
        'SuperPower': {'S': 'Super intelligence'},
        'SuperWeapon': {'S': 'Mark 50'},
        'Villain1': {'S': 'Mandarin'},
        'Villain2': {'S': 'Whiplash'},
        'Villain3': {'S': 'Ultron'},
    }
)
dynamo.put_item(
    TableName=table_name,
    Item={
        'SuperHero': {'S': 'Captain America'},
        'SuperMission': {'S': 'Save the world'},
        'SuperPower': {'S': 'Super strength'},
        'SuperWeapon': {'S': 'Shield'},
        'Villain1': {'S': 'Red Skull'},
        'Villain2': {'S': 'Hydra'},
    }
)

{'ResponseMetadata': {'RequestId': '4GNVAM4FJ898RJFE2JAORLDBHVVV4KQNSO5AEMVJF66Q9ASUAAJG',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Sun, 16 Oct 2022 22:39:17 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-amzn-requestid': '4GNVAM4FJ898RJFE2JAORLDBHVVV4KQNSO5AEMVJF66Q9ASUAAJG',
   'x-amz-crc32': '2745614147'},
  'RetryAttempts': 0}}

## Revisar las políticas y los roles de IAM

In [14]:
policy_table = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Action": [
                "dynamodb:Scan",
                "s3:GetObject",
                "s3:PutObject",
                "dynamodb:BatchWriteItem"
            ],
            "Resource": [
                "*"
            ],
            "Effect": "Allow"
        }
    ]
}

Esta es una política sencilla que concede acceso a las API de análisis, de BatchWriteItem en DynamoDB y las API de GetObject y de PutObject en S3 de su cuenta.

In [26]:
# Create a policy
policy_scan_name = 'Super-DynamoDBScanPolicy'
response = iam.create_policy(
    PolicyName= policy_scan_name,
    PolicyDocument=json.dumps(policy_table)
)

In [27]:
# Get the ARN from the policy
policy_scan_arn = response['Policy']['Arn']

In [20]:
policy_query = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Condition": {
                "ForAllValues:StringEquals": {
                    "dynamodb:Attributes": [
                        "SuperHero",
                        "MissionStatus",
                        "Villain1",
                        "Villain2",
                        "Villain3"
                    ]
                }
            },
            "Action": [
                "dynamodb:Query"
            ],
            "Resource": "*",
            "Effect": "Allow"
        }
    ]
}

Esta política permite al usuario o entidad que asume el rol realizar una operación de consulta, pero solo con los atributos especificados. Esta función útil le permite implementar seguridad a nivel de las columnas en sus tablas de DynamoDB. Utilizará este rol en el siguiente laboratorio para habilitar una función de AWS Lambda para leer datos de esta tabla.

In [23]:
# Create a policy
policy_query_name = 'SuperDynamoDBQueryPolicy'
response = iam.create_policy(
    PolicyName= policy_query_name,
    PolicyDocument=json.dumps(policy_query)
)

In [25]:
# Get the ARN from the policy
policy_query_arn = response['Policy']['Arn']

In [32]:
# Create SuperDynamoDBRole role
role_name = 'SuperDynamoDBRole'
assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Action": "sts:AssumeRole",
            "Principal": {
                "Service": "lambda.amazonaws.com"
            },
            "Effect": "Allow"
        }
    ]
}
response = iam.create_role(
    RoleName=role_name,
    AssumeRolePolicyDocument=json.dumps(assume_role_policy_document)
)

{'Role': {'Path': '/',
  'RoleName': 'SuperDynamoDBRole',
  'RoleId': 'AROAWEYWGFCJOIU2ZHBOC',
  'Arn': 'arn:aws:iam::422563948690:role/SuperDynamoDBRole',
  'CreateDate': datetime.datetime(2022, 10, 16, 21, 36, 38, tzinfo=tzutc()),
  'AssumeRolePolicyDocument': {'Version': '2012-10-17',
   'Statement': [{'Action': 'sts:AssumeRole',
     'Principal': {'Service': 'lambda.amazonaws.com'},
     'Effect': 'Allow'}]}},
 'ResponseMetadata': {'RequestId': 'e71827fd-6f9c-4ac3-80bf-1ab5a6bffc1c',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'e71827fd-6f9c-4ac3-80bf-1ab5a6bffc1c',
   'content-type': 'text/xml',
   'content-length': '788',
   'date': 'Sun, 16 Oct 2022 21:36:37 GMT'},
  'RetryAttempts': 0}}

In [38]:
role_super_dynamo_arn = response['Role']['Arn']

In [ ]:
# Create SuperDynamoDBRole role
role_name = 'SuperDynamoDBRole'
assume_role_policy_document = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Action": "sts:AssumeRole",
            "Principal": {
                "Service": "lambda.amazonaws.com"
            },
            "Effect": "Allow"
        }
    ]
}
response = iam.create_role(
    RoleName=role_name,
    AssumeRolePolicyDocument=json.dumps(assume_role_policy_document)
)

In [40]:
# Create SuperDynamoDbQueryRole role
role_name_query = 'SuperDynamoDbQueryRole'
response = iam.create_role(
    RoleName=role_name_query,
    AssumeRolePolicyDocument=json.dumps(assume_role_policy_document)
)

In [41]:
role_super_dynamo_query_arn = response['Role']['Arn']

In [42]:
# Attach policy to role
response = iam.attach_role_policy(
    RoleName=role_name,
    PolicyArn=policy_scan_arn
)

response = iam.attach_role_policy(
    RoleName=role_name_query,
    PolicyArn=policy_query_arn
)

## Crear las funciones Lambda

### Primera Función

In [50]:
lambda_name = "SuperDynamoDBFunction"
lambda_py = lambda_name + ".py"
file_lambda = "lambda.zip"

In [48]:
%%writefile SuperDynamoDBFunction.py
import boto3
import json

dynamodb = boto3.resource('dynamodb')
table = dynamodb.Table('SuperMissionTable')

def lambda_handler(event, context):
    response = table.scan()
    return response['Items']

Writing SuperDynamoDBFunction.py


In [52]:
with ZipFile(file_lambda, 'w') as myzip:
    myzip.write(lambda_py)

with open(file_lambda, 'rb') as f:
    zipped_code = f.read()

In [54]:
# Create a lambda function
response = lambda_client.create_function(
    FunctionName = lambda_name,
    Runtime = 'python3.9',
    Role = role_super_dynamo_arn,
    Handler = lambda_name + '.lambda_handler',
    Code = {'ZipFile': zipped_code},
    Timeout = 90,
    Environment = {
        'Variables': {
            'Name': lambda_name,
            'Environment': 'prod'}
    },
    Publish=True
)

In [87]:
# Test the lambda function
response = lambda_client.invoke(
    FunctionName='SuperDynamoDBFunction',
    InvocationType='RequestResponse',
    Payload=json.dumps({"superhero": "Superman"})
)

json_data = json.loads(response['Payload'].read())
pprint(json_data)

[{'SuperHero': 'Captain America',
  'SuperMission': 'Save the world',
  'SuperPower': 'Super strength',
  'SuperWeapon': 'Shield',
  'Villain1': 'Red Skull',
  'Villain2': 'Hydra'},
 {'SuperHero': 'Batman',
  'SuperMission': 'Save Gotham City',
  'SuperPower': 'Super intelligence',
  'SuperWeapon': 'Utility belt',
  'Villain1': 'Joker',
  'Villain2': 'Two-Face',
  'Villain3': 'Penguin'},
 {'SuperHero': 'Spiderman',
  'SuperMission': 'Catch the bad guys',
  'SuperPower': 'Super agility',
  'SuperWeapon': 'Spider web',
  'Villain1': 'Green Goblin',
  'Villain2': 'Venom'},
 {'SuperHero': 'Ironman',
  'SuperMission': 'Save the world',
  'SuperPower': 'Super intelligence',
  'SuperWeapon': 'Mark 50',
  'Villain1': 'Mandarin',
  'Villain2': 'Whiplash',
  'Villain3': 'Ultron'},
 {'SuperHero': 'Superman',
  'SuperMission': 'Save the world',
  'SuperPower': 'Super strength',
  'SuperWeakness': 'Kryptonite',
  'SuperWeapon': 'Laser eyes',
  'Villain1': 'Lex Luthor',
  'Villain2': 'Brainiac',
  '

### Segunda Función

In [55]:
lambda_name = "MissionDetailsFunction"
lambda_py = lambda_name + ".py"
file_lambda = "lambda.zip"


In [60]:
%%writefile MissionDetailsFunction.py
import boto3
from boto3.dynamodb.conditions import Key, Attr
import json

def lambda_handler(event, context):
    dynamodb = boto3.resource('dynamodb')
    table = dynamodb.Table('SuperMissionTable')

    condition = {}
    condition["SuperHero"] = event["superhero"]

    response = table.query(KeyConditionExpression=Key("SuperHero").eq(condition["SuperHero"]))
    return response


Overwriting MissionDetailsFunction.py


In [61]:
with ZipFile(file_lambda, 'w') as myzip:
    myzip.write(lambda_py)

with open(file_lambda, 'rb') as f:
    zipped_code = f.read()

In [62]:
# Create a lambda function
response = lambda_client.create_function(
    FunctionName = lambda_name,
    Runtime = 'python3.9',
    Role = role_super_dynamo_query_arn,
    Handler = lambda_name + '.lambda_handler',
    Code = {'ZipFile': zipped_code},
    Timeout = 90,
    Environment = {
        'Variables': {
            'Name': lambda_name,
            'Environment': 'prod'}
    },
    Publish=True
)

In [88]:
# Test the lambda function
response = lambda_client.invoke(
    FunctionName='MissionDetailsFunction',
    InvocationType='RequestResponse',
    Payload=json.dumps({"superhero": "Superman"})
)

json_data = json.loads(response['Payload'].read())
pprint(json_data)

{'Count': 1,
 'Items': [{'SuperHero': 'Superman',
            'SuperMission': 'Save the world',
            'SuperPower': 'Super strength',
            'SuperWeakness': 'Kryptonite',
            'SuperWeapon': 'Laser eyes',
            'Villain1': 'Lex Luthor',
            'Villain2': 'Brainiac',
            'Villain3': 'Darkseid',
            'Villain4': 'Bizarro'}],
 'ResponseMetadata': {'HTTPHeaders': {'connection': 'keep-alive',
                                      'content-length': '324',
                                      'content-type': 'application/x-amz-json-1.0',
                                      'date': 'Sun, 16 Oct 2022 22:47:10 GMT',
                                      'server': 'Server',
                                      'x-amz-crc32': '3067554583',
                                      'x-amzn-requestid': 'IR9E84AS1791023J51O8K2KUR3VV4KQNSO5AEMVJF66Q9ASUAAJG'},
                      'HTTPStatusCode': 200,
                      'RequestId': 'IR9E84AS1791023J

In [89]:
response = lambda_client.invoke(
    FunctionName='MissionDetailsFunction',
    InvocationType='RequestResponse',
    Payload=json.dumps({"superhero": "Ironman"})
)

json_data = json.loads(response['Payload'].read())
pprint(json_data)

{'Count': 1,
 'Items': [{'SuperHero': 'Ironman',
            'SuperMission': 'Save the world',
            'SuperPower': 'Super intelligence',
            'SuperWeapon': 'Mark 50',
            'Villain1': 'Mandarin',
            'Villain2': 'Whiplash',
            'Villain3': 'Ultron'}],
 'ResponseMetadata': {'HTTPHeaders': {'connection': 'keep-alive',
                                      'content-length': '258',
                                      'content-type': 'application/x-amz-json-1.0',
                                      'date': 'Sun, 16 Oct 2022 22:47:25 GMT',
                                      'server': 'Server',
                                      'x-amz-crc32': '2296869215',
                                      'x-amzn-requestid': 'MK380GP305OQ9TMRLDKN2LQCCJVV4KQNSO5AEMVJF66Q9ASUAAJG'},
                      'HTTPStatusCode': 200,
                      'RequestId': 'MK380GP305OQ9TMRLDKN2LQCCJVV4KQNSO5AEMVJF66Q9ASUAAJG',
                      'RetryAttempts': 0},
